In [30]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import LineString
import sys

sys.path.append('../')
import road
from elevation import get_elevation_from_srtm, calc_incline

In [31]:
bbox =(48.7861400822045,2.197526708826558,48.93467868121917,2.4458831002388592)
#bbox = (45.45, -73.6492855702676, 45.644709880535515, -73.44752816469271)
#bbox='<BB>'
highway_list = ["motorway", "motorway_link", "trunk", "trunk_link", "primary", "primary_link", 
                  "secondary", "secondary_link", "tertiary", "tertiary_link", "residential","cycleway"]
#ighway_list = ["motorway", "motorway_link", "trunk", "trunk_link", "primary", "primary_link"]
cycleway_list = ["lane", "opposite", "opposite_lane", "track", "opposite_track", 
                 "share_busway", "opposite_share_busway", "shared_lane", "lane"]

In [32]:
#overpass_query += ''f'way["bicycle_road"="yes"]{bbox};\n


In [33]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query ="""
[out:json][timeout:180];
(
"""
overpass_query += ''.join([f'way["highway"="{highway}"]{bbox};\n' for highway in highway_list])
overpass_query += ''.join([f'way["cycleway"="{cycleway}"]{bbox};\n' for cycleway in cycleway_list])
overpass_query +=""" 
);
out body;
>;
out skel qt;
"""

response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

In [34]:
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

In [35]:
"""
[out:json][timeout:180][bbox:{{bbox}}];
(
way["highway"="residential"]; // find all nodes in current bounding box with start_date tag
);
convert way 
    ::id = id(),
	"nodes" = geometry, // Should be implemened in futur version
	"oneway" = t["oneway"];
out body;
>;
out skel qt;
"""

'\n[out:json][timeout:180][bbox:{{bbox}}];\n(\nway["highway"="residential"]; // find all nodes in current bounding box with start_date tag\n);\nconvert way \n    ::id = id(),\n\t"nodes" = geometry, // Should be implemened in futur version\n\t"oneway" = t["oneway"];\nout body;\n>;\nout skel qt;\n'

overpass_url = "http://overpass-api.de/api/interpreter"
data = []
for highway in highway_list:
    overpass_query ="""
    [out:json][timeout:180];
    (
    """
    overpass_query += ''.join(f'way["highway"={highway}]{bbox};\n')
    overpass_query +=""" 
    );
    out body;
    >;
    out skel qt;
    """
    data.append(overpass_query)


import asyncio
import aiohttp
import time

async def get(url, data, session):
    try:
        async with session.get(url=url, data=data) as response:
            resp = await response.json()
            print("Successfully got url {} with resp of length {}.".format(url, len(resp)))
            return await response
    except Exception as e:
        print("Unable to get url {} due to {}.".format(url, e.__class__))


async def main(url, data):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[get(url, d, session) for d in data])
    print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))
    return ret

response = await main(overpass_url, data)

In [36]:
way = pd.DataFrame([d for d in data['elements'] if d['type'] == 'way']).set_index('id')
nodes = pd.DataFrame([d for d in data['elements'] if d['type'] == 'node']).set_index('id')

way_exploded = way.explode('nodes').merge(nodes[['lat','lon']], left_on='nodes', right_index=True, how='left')

geom = way_exploded.groupby('id')[['lon', 'lat']].apply(lambda x: LineString(x.values))
geom.name = 'geometry'
way = gpd.GeoDataFrame(way.join(geom), crs=4326)


/home/boivin/.pyenv/versions/quetzal_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [37]:
tags = pd.DataFrame.from_records(way['tags'].values, index=way['tags'].index)
cols = ['highway', 'maxspeed', 'lanes', 'name', 'oneway', 'surface']
cols += ['cycleway','cycleway:both', 'cycleway:left','cycleway:right']

In [38]:
cols

['highway',
 'maxspeed',
 'lanes',
 'name',
 'oneway',
 'surface',
 'cycleway',
 'cycleway:both',
 'cycleway:left',
 'cycleway:right']

In [39]:
cycle_cols = []
for col in tags.columns:
    if 'cycle' in col:
        cycle_cols.append(col)
#cols = cols + cycle_cols
cycle_cols

['cycleway:right',
 'cycleway',
 'cycleway:left',
 'cycleway:left:oneway',
 'oneway:bicycle',
 'cycleway:both',
 'bicycle',
 'bicycle:backward',
 'cycleway:right:lane',
 'source:oneway:bicycle',
 'bicycle:lanes:backward',
 'cycleway:start_date',
 'cycleway:right:segregated',
 'bicycle:lanes:forward',
 'was:cycleway:right',
 'bicycle:lanes',
 'cycleway:left:lane',
 'cycleway:temporary',
 'cycleway:both:lane',
 'source:cycleway',
 'check_date:cycleway',
 'cycleway:right:width',
 'class:bicycle',
 'motorcycle',
 'cyclestreet',
 'cycleway:lane',
 'cycleway:right:temporary',
 'was:cycleway:left',
 'cycleway:right:oneway',
 'maxspeed:bicycle',
 'was:cycleway',
 'sidewalk:right:bicycle',
 'temporary:cycleway:left',
 'cycleway:surface',
 'cycleway_link',
 'bicycle_road',
 'cycleway:width',
 'cycleway:lanes',
 'note:bicycle',
 'was:bicycle',
 'cycleway:left:temporary',
 'was:oneway:bicycle',
 'cycleway:right:track',
 'cyclestreets_id',
 'cycleway:est_width',
 'sidewalk:left:bicycle',
 'sidewalk

In [12]:
#https://wiki.openstreetmap.org/wiki/Map_features#When_cycleway_is_drawn_as_its_own_way_(see_Bicycle)

In [13]:
way_tags = way.drop(columns=['nodes', 'tags'], errors='ignore').join(tags[cols])

In [14]:
# OSMNX add edges speed, length and time
# garder les liens fortement connexes
# retirer les noeuds de degree 2

In [15]:
# SOME CLEANING ON THE ONEWAY ... Work In Progress
way_tags['oneway'].fillna('no', inplace=True)
way_tags['oneway'] = way_tags['oneway'].replace('yes', True).replace('no', False).replace('-1', False).replace(-1, False).replace('alternating',False)


In [501]:
way_tags.to_file('way.geojson',driver='GeoJSON')

In [502]:
links, nodes = road.get_links_and_nodes('way.geojson', split_direction=False)
nodes = nodes.set_crs(links.crs)

In [503]:
print(len(way_tags))
print(len(links))

24283
40151


In [504]:
links = road.clean_maxspeed(links)

In [505]:
def rename_bicycle_tags(df,col) :
    links=df.copy()
    print(col)
    print(list(links[col].unique()))
    links['agg_'+col] = links[col]
    # fill NaN with no
    links.loc[links['agg_'+col].astype('str')=='nan','agg_'+col] = 'no'
    cycle_dict={'no':'no', 
                'shared':'shared',
                'share_busway':'shared', 
                'shared_lane':'shared'}

    links['agg_'+col] = links['agg_'+col].apply(lambda x: cycle_dict.get(x,'yes'))
    print(list(links['agg_'+col].unique()))
    return links

In [506]:
links = rename_bicycle_tags(links,'cycleway')

cycleway
[nan, 'lane', 'shared_lane', 'no', 'track', 'crossing', 'cyclestreet', 'share_busway', 'shared', 'opposite_lane', 'use_sidepath', 'separate']
['no', 'yes', 'shared']


In [507]:
links = rename_bicycle_tags(links,'cycleway:both')

cycleway:both
['no', nan, 'separate', 'lane', 'shared_lane', 'share_busway', 'track']
['no', 'yes', 'shared']


In [508]:
links = rename_bicycle_tags(links,'cycleway:left')

cycleway:left
[nan, 'opposite_lane', 'lane', 'no', 'track', 'opposite_track', 'shared_lane', 'separate', 'share_busway', 'none']
['no', 'yes', 'shared']


In [509]:
links = rename_bicycle_tags(links,'cycleway:right')

cycleway:right
[nan, 'lane', 'no', 'shared_lane', 'track', 'separate', 'share_busway', 'cyclestreet']
['no', 'yes', 'shared']


In [510]:
links[['agg_cycleway', 'agg_cycleway:both','agg_cycleway:left','agg_cycleway:right']]

,agg_cycleway,agg_cycleway:both,agg_cycleway:left,agg_cycleway:right
road_link_0,no,no,no,no
road_link_1,no,no,no,no
road_link_2,no,no,no,no
road_link_3,no,no,no,no
road_link_4,no,no,no,no
...,...,...,...,...
road_link_40146,no,no,no,no
road_link_40147,no,no,no,no
road_link_40148,no,no,no,no
road_link_40149,yes,no,no,no


In [511]:
links['combine_cycle_tag'] = links['agg_cycleway'] +' '+ links['agg_cycleway:both'] +' '+  links['agg_cycleway:left'] +' '+ links['agg_cycleway:right'] 

In [512]:
print('c, b, l, r')
for cal in links['combine_cycle_tag'].unique():print(cal)

c, b, l, r
no no no no
yes no no no
no no yes yes
shared no no no
no no yes shared
no no no yes
no yes no no
no no yes no
no no no shared
no shared no no
yes yes no no
no no shared yes
no no shared no
no no shared shared
no yes yes shared
shared no shared shared
no shared yes shared
yes no no shared
no yes no yes


In [513]:
links.to_file('b_links.geojson',drivers='GeoJSON')
nodes.to_file('b_nodes.geojson',drivers='GeoJSON')

In [270]:
links = road.rectify_geometry_direction(links,nodes)

0 geometry to inverse


In [271]:
links = road.drop_duplicated_links(links)

121 links dropped


In [272]:
len(links)

24520

In [273]:
links = road.simplify(links)
len(links)

11680 deg 2 nodes
find path with large cutoff for 1  origins
find path with large cutoff for 2  origins
find path with large cutoff for 1  origins
0 links were not merge because the oneway field is not the same
0 links were not merge because the highway field is not the same
0 merged_links unmerged because the geometry became a multilinestring


21251

In [274]:
links = road.split_oneway(links)
len(links)

33179

In [275]:
links = road.main_strongly_connected_component(links,None,False)
len(links)

32615

In [276]:
def process_list_in_col(col_values,new_type,function):
    if isinstance(col_values, list):
        return  function([new_type(val) for val in col_values])
    else:
        return new_type(col_values)
    
def remove_list_in_col(col_values,method='first'):
    if isinstance(col_values, list):
        if method == 'first':
            return col_values[0]
        else:
            return col_values[-1]
        
    else:
        return col_values
    
links['maxspeed'] = links['maxspeed'].apply(lambda x: process_list_in_col(x,float,np.nanmean))
links['lanes'] = links['lanes'].apply(lambda x: process_list_in_col(x,float,np.nanmean)).apply(lambda x: np.floor(x))

/tmp/ipykernel_591767/3507984463.py:3: RuntimeWarning: Mean of empty slice
  return  function([new_type(val) for val in col_values])


In [277]:
for col in ['id', 'type', 'highway','name','surface']:
    links[col] = links[col].apply(lambda x: remove_list_in_col(x,'first'))

In [278]:
links['length'] = links.to_crs(32618).length


In [279]:



    
links.loc[~links['maxspeed'].astype(str).str.isdigit(),'maxspeed']=np.nan
links['maxspeed'] = pd.to_numeric(links['maxspeed'])
speed_dict = links.dropna().groupby('highway')['maxspeed'].agg(np.mean).to_dict()
links.loc[~np.isfinite(links['maxspeed']),'maxspeed'] = links.loc[~np.isfinite(links['maxspeed']),'highway'].apply(lambda x: speed_dict.get(x))
links['time'] = links['length']/(links['maxspeed']*1000/3600)

In [280]:
links['lanes'] = pd.to_numeric(links['lanes'])
lane_dict = links.groupby('highway')['lanes'].agg(np.nanmean).apply(lambda x: np.floor(x)).to_dict()
links.loc[~np.isfinite(links['lanes']),'lanes'] = links.loc[~np.isfinite(links['lanes']),'highway'].apply(lambda x: lane_dict.get(x))


In [281]:
links = links.drop(columns=['id','type','oneway','surface'])
links.geometry = links.simplify(0.00003)

In [282]:
links = links.reset_index(drop=True)
links.index = 'road_link_'+links.index.astype(str)

nodes_set = set(links['a']).union(set(links['b']))

nodes = nodes.loc[nodes_set].sort_index()


/tmp/ipykernel_591767/722026835.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  nodes = nodes.loc[nodes_set].sort_index()


In [283]:
links = road.drop_duplicated_links(links)


325 links dropped


# Elevation

In [29]:
el_dict = get_elevation_from_srtm(nodes)

file save tmp/


In [30]:
nodes['elevation'] = nodes.index.map(el_dict.get)

In [31]:
# incline from node a to b in deg. neg if going down (if b is lower dans a)
elevation_dict = nodes['elevation'].to_dict()
links['incline'] = calc_incline(links['a'].apply(lambda x: el_dict.get(x)).values,
                               links['b'].apply(lambda x: el_dict.get(x)).values,
                               links['length'].values)

In [34]:
links['incline'].min()

-7.025106537874201

In [90]:
links.to_file('road_links.geojson')
nodes.to_file('road_nodes.geojson')